# *Fruit and Vegetables- CNN Model*

*Bu projede, görüntüleri analiz ederek türüne göre meyve ya da sebze diye sınıflandıran bir CNN Modeli geliştireceğiz. Modeli kaydedip Streamlit uygulamasına çevireceğiz ve HugginFace de çalışır hale getireceğiz.*

### *import Libraries*

In [1]:
import os                  # Dosya ve dizin işlemleri için
import random              # Rastgele seçimler yapmak için
import warnings            # Uyarı mesajlarını yönetmek için
warnings.filterwarnings('ignore') # Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz

import numpy as np         # Sayısal hesaplamalar ve matris işlemleri
import pandas as pd        # Veri analizi ve tablo işlemleri
import cv2                 # Görüntü işleme ve video/frame işlemleri
import kagglehub           # Kaggle veri setlerini kolayca çekmek için
import random

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split  # Veriyi eğitim/test setlerine ayırmak için

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model            # Keras Functional API model tanımı
from tensorflow.keras.layers import Flatten, Dense   # Katmanlar: Flatten ve tam bağlantılı Dense
from tensorflow.keras.applications import VGG16      # Önceden eğitilmiş VGG16 modeli
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


### *İmport Data*

In [2]:
# Görüntülerin bulunduğu ana dizin yolunu belirtiyoruz
# Download latest version
path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")

print("Path to dataset files:", path)

100%|██████████| 1.98G/1.98G [01:32<00:00, 23.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kritikseth/fruit-and-vegetable-image-recognition/versions/8


In [3]:
img_path = os.path.join(path)
os.listdir(img_path)

['validation', 'train', 'test']

### *CNN with Transfer Learning*

In [4]:
# Resim boyutlarını ayarlayın
img_width, img_height = 224, 224

# Veri dizinini tanımlayın
img_path = kagglehub.dataset_download("kritikseth/fruit-and-vegetable-image-recognition")   # Burada doğru dizini belirtmelisiniz

# Eğitim ve doğrulama veri dizinleri
train_dir = os.path.join(img_path, "train")  # Eğitim verisi dizini
validation_dir = os.path.join(img_path, "validation")  # Doğrulama verisi dizini

# Eğitim ve doğrulama verilerini hazırlamak için ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Eğitim veri kümesini yükleyin
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Eğitim verisi dizini
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',  # Çoklu sınıf problemi
    subset='training'  # Eğitim verisi
)

# Doğrulama veri kümesini yükleyin
validation_generator = train_datagen.flow_from_directory(
    validation_dir,  # Doğrulama verisi dizini
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',  # Çoklu sınıf problemi
    subset='validation'  # Doğrulama verisi
)


Found 2510 images belonging to 36 classes.
Found 63 images belonging to 36 classes.


In [5]:
# VGG16 modelini yükleyin (ImageNet'ten önceden eğitilmiş)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False  # Önceden eğitilmiş katmanları dondur

# Üst katmanları ekleyin
x = base_model.output
x = Flatten()(x)  # Çıkışı düzleştir
x = Dense(1024, activation='relu')(x)  # Tam bağlantılı katman
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # Sınıf sayısı kadar çıktı

# Modeli tanımlayın (Functional API kullanılarak)
model = Model(inputs=base_model.input, outputs=predictions)

# Modeli derleyin
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitin
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.1954 - loss: 6.0987 - val_accuracy: 0.8889 - val_loss: 0.5973
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 63s 798ms/step - accuracy: 0.8095 - loss: 0.7462 - val_accuracy: 0.9048 - val_loss: 0.3416
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 62s 790ms/step - accuracy: 0.9448 - loss: 0.2554 - val_accuracy: 0.9048 - val_loss: 0.2889
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 64s 806ms/step - accuracy: 0.9867 - loss: 0.1035 - val_accuracy: 0.9365 - val_loss: 0.2165
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 64s 813ms/step - accuracy: 0.9898 - loss: 0.0682 - val_accuracy: 0.9524 - val_loss: 0.1595
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 63s 804ms/step - accuracy: 0.9909 - loss: 0.0576 - val_accuracy: 0.9524 - val_loss: 0.1586
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 62s 786ms/step - accuracy: 0.9933 - loss: 0.0372 - val_accuracy: 0.9524 - val_loss: 0.1457
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 64s 810ms/

### *Save the Model*

In [7]:
# Modeli kaydedin
model.save('fruits_and_vegetables_TL.keras')

*Bu projede meyve ve sebze görüntülerini analiz ederek türlerini sınıflandıran bir CNN Modeli geliştirdik. Datayı kaggle üzerinden çektik. Kendi datamızı önceden eğitilmiş VGG16 modeli ile birleştirip geliştirdik. %99.46 başarı elde ettik.*